## HSP90 with ADP:Mg2+ simulation
* https://openmm.github.io/openmm-cookbook/latest/notebooks/tutorials/HSP90_with_ADPMg2_simulation.html#HSP90-with-ADP:Mg2+-simulation
* Simulating a protein:ligand complex with multisite solvated ions.

### TOC
* We begin from the 1BYQ PDB file, add missing residues (only those in the middle of the chain) and missing heavy atoms using PDBFixer. 
* Using MDTraj, residue and atom naming of the Mg2+ ion and the ADP are fixed to match those in the parameter files, and dummy atoms of the multisite Mg2+ are added. 
* CONECT records for bonds between Mg2+ and ADP are deleted. Crystallographic waters are preserved for coordination to the Mg2+.
* Finally, tleap is run to add hydrogens and parametrize. 

### Details
* We use the ff99SBildn force field, mag.lib and frcmod_mg.mmg files from the multisite Mg2+ model (downloaded from Sept Lab), ADP.prep and frcmod.phos files for the ADP parameters (downloaded from Bryce group Amber parameter database). 
* The prmtop and inpcrd files are saved for simulation in OpenMM. * 
The multisite Mg2+ model requires a correction to the Lennard-Jones B-coefficients in the prmtop file, this is done by a short Python script (obtained by email from Prof. David Sept, Sept Lab).

In [83]:
# Download the files
# You need to run this cell only once at the first time you run this notebook
#!wget https://openmm.org/tutorials_/hsp90_adp_mg/files/hsp90_adp_mg.zip -o ./data/hsp90_adp_mg.zip
#!unzip -o hsp90_adp_mg.zip -d ./data/hsp90_adp_mg

In [84]:
from pdbfixer import PDBFixer
from openmm.app import PDBFile
import mdtraj as md
import os
import numpy as np

In [85]:
# clean up the original PDB file and add missing residues and heavy atoms
fixer = PDBFixer('./data/hsp90_adp_mg/pdb1byq.ent') # it contain pbd.


In [86]:
import nglview as nv
view = nv.show_structure_file('./data/hsp90_adp_mg/pdb1byq.ent')
view

NGLWidget()

In [87]:
fixer.findMissingResidues()
fixer.missingResidues

{(0, 0): ['ASP', 'GLN'],
 (0, 213): ['LYS',
  'GLU',
  'ARG',
  'ASP',
  'LYS',
  'GLU',
  'VAL',
  'SER',
  'ASP',
  'ASP',
  'GLU',
  'ALA',
  'GLU']}

In [88]:
chains = list(fixer.topology.chains())
print(chains)

[<Chain 0>, <Chain 1>]


In [89]:
keys = fixer.missingResidues.keys()
keys

dict_keys([(0, 0), (0, 213)])

In [90]:
missingResidues = dict()
for key in keys:
    chain = chains[key[0]]
    if not (key[1] == 0 or key[1] == len(list(chain.residues()))):
        # ignore terminal residues(0 and last)
        missingResidues[key] = fixer.missingResidues[key]
        # since we have only termianl missing residues in the 0 chain.
        # we can ignore the missing residues
fixer.missingResidues = missingResidues
print(fixer.missingResidues)

{}


In [91]:
fixer.findMissingAtoms()
fixer.addMissingAtoms()

In [92]:
PDBFile.writeFile(fixer.topology, fixer.positions, open('./data/hsp90_adp_mg/1byq_fixed.pdb', 'w'))

In [93]:
view= nv.show_structure_file('./data/hsp90_adp_mg/1byq_fixed.pdb')
view.add_representation('licorice', selection='water')
view.add_representation('spacefill', selection='ion')
view

NGLWidget()

In [94]:
traj = md.load('./data/hsp90_adp_mg/1byq_fixed.pdb')

In [95]:

# fix residue names
# in adp - change ` to * in atom names
# add Mg dummy atoms
#  Using MDTraj, residue and atom naming of the Mg2+ ion and the ADP are fixed to match those in the parameter files, and dummy atoms of the multisite Mg2+ are added. 
for residue in traj.top.residues:
    if residue.name == 'MG': # if it is a magnesium ion
        residue.name = 'MMG' # rename it to MMG
        residue.atom(0).name = 'XZ' # rename the atom to XZ
        for i in range(6):
            traj.top.add_atom(f'D{i+1}', md.element.Element.getBySymbol('Mg'), residue)
        # add dummy atoms
        central_index = residue.atom(0).index # index of the central atom
        central_positions = traj.xyz[0,central_index] # position of the central atom
        # positions from magnesium_ms.pdb, Sept Lab: http://septlab.engin.umich.edu/multisite-ions.html
        dummy_positions = central_positions + [
            [0.0, 0.0, 0.09], 
            [0.09, 0.0, 0.0], 
            [0.0, 0.0, -0.09],
            [-0.09, 0.0, 0.0],
            [0.0, -0.09, 0.0],
            [0.0, 0.09, 0.0]
        ]
        traj.xyz = np.array([np.insert(traj.xyz[0], central_index+1, dummy_positions, axis=0)])
    elif residue.name == 'ADP': # if it is an adenosine diphosphate
        residue.name = 'adp' # rename it to adp
        for atom in residue.atoms:
            if atom.name[-1] == "'": # if it is a prime atom
                atom.name = atom.name[:-1] + "*" # remove prime and add asterisk

In [96]:
# remove adp - mg CONECT bonds
bonds = []
for bond in traj.top._bonds:
    if not (bond[0].residue.name == 'MMG' or bond[1].residue.name == 'MMG'):
        # remove MMG bonds
        bonds.append(bond)
traj.top._bonds = bonds # update bonds

In [97]:
traj.save('./data/hsp90_adp_mg/1byq_fixed2.pdb')

In [98]:
view = nv.show_structure_file('./data/hsp90_adp_mg/1byq_fixed2.pdb')
view.add_representation('licorice', selection='water')
view.add_representation('spacefill', selection='ion')
view

NGLWidget()

In [99]:
# * Finally, tleap is run to add hydrogens and parametrize. 
# save the tleap script to file
with open('./data/hsp90_adp_mg/leaprc.hsp90', 'w') as f:
    f.write('''
source oldff/leaprc.ff99SBildn
loadOff ./data/hsp90_adp_mg/mag.lib
loadamberparams ./data/hsp90_adp_mg/frcmod_mg.mmg
loadAmberPrep ./data/hsp90_adp_mg/ADP.prep
loadamberparams ./data/hsp90_adp_mg/frcmod.phos
x = loadPdb ./data/hsp90_adp_mg/1byq_fixed2.pdb
addIons x Na+ 0
solvateBox x TIP3PBOX 10.0
savePdb x ./data/hsp90_adp_mg/topology.pdb
saveAmberParm x ./data/hsp90_adp_mg/input.prmtop ./data/hsp90_adp_mg/input.inpcrd
quit
''')

In [100]:
#os.system('tleap -f ./data/hsp90_adp_mg/leaprc.hsp90')
# don't run tleap, just use the prepared files

In [101]:
from openmm import app
import openmm as mm
from openmm import unit
from sys import stdout

In [102]:
# load in Amber input files
prmtop = app.AmberPrmtopFile('./data/hsp90_adp_mg/input.prmtop.mod')
inpcrd = app.AmberInpcrdFile('./data/hsp90_adp_mg/input.inpcrd')

In [103]:
# prepare system and integrator

system = prmtop.createSystem(nonbondedMethod=app.PME,
    nonbondedCutoff=1.0*unit.nanometers, constraints=app.HBonds, rigidWater=True,
    ewaldErrorTolerance=0.0005)
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds,
    2.0*unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

In [104]:
# prepare simulation
platform = mm.Platform.getPlatformByName('CUDA')
simulation = app.Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)

In [105]:
# minimize
print('Minimizing...')
simulation.minimizeEnergy()

Minimizing...


In [106]:
# equilibrate for 100 steps
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
print('Equilibrating...')
simulation.step(100)

Equilibrating...


In [107]:
# append reporters
simulation.reporters.append(app.DCDReporter('./results/HSP90_trajectory.dcd', 1000))
simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True,
    potentialEnergy=True, temperature=True, progress=True, remainingTime=True,
    speed=True, totalSteps=250000, separator='\t'))

In [108]:
# run 0.5 ns of production simulation, it takes about 8 min
# for 50 ns, it takes about 12 hrs
print('Running Production...')
simulation.step(250000)
print('Done!')

Running Production...
#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
0.4%	1000	-657769.6012285054	268.2016723165408	0	--
0.8%	2000	-636196.8570878804	293.22729685550223	85.6	8:20
1.2%	3000	-631532.6656816304	298.6068512882197	90.5	7:51
1.6%	4000	-631483.1344316304	298.8572335229811	89.2	7:56
2.0%	5000	-631041.3805253804	298.97435032435277	90.7	7:46
2.4%	6000	-633123.4781816304	301.93801629793006	89.7	7:49
2.8%	7000	-631609.2555253804	298.4986612869753	90.5	7:43
3.2%	8000	-632389.1969316304	300.099004674429	89.7	7:45
3.6%	9000	-635907.8063066304	301.3292086179184	90.3	7:41
4.0%	10000	-635021.1207597554	301.04010130641404	90.6	7:37
4.4%	11000	-636274.3902910054	300.3470966437173	90	7:38
4.8%	12000	-637817.6852128804	298.3449565106069	90.3	7:35
5.2%	13000	-640415.3375566304	302.469646637822	89.7	7:36
5.6%	14000	-637471.2223222554	300.32685241106515	89.9	7:33
6.0%	15000	-638573.0172441304	300.1925554546667	90.1	7:30
6.4%	16000	-63876

In [109]:
## we should revist this code book to analyse the results